## Recuit simulé pour le problème du voyageur de commerce

CSI4506 Intelligence Artificielle  
Automne 2018  
Caroline Barrière

Ce devoir vous permettra de mieux comprendre les algorithmes de recherche aléatoire vus en classe, en particulier l'algorithme de *recuit simulé*, qui est une méthode connue qui exemplifie bien la *modification aléatoire*.  Au départ, une solution gloutonne est proposée.  Cette solution est révisée petit à petit pour arriver à une meilleure solution.

Le domaine d'application utilisé ici pour expérimenter avec le *recuit simulé* est le problème du voyageur de commerce (Traveling salesman problem).  C'est un problème classique, dans lequel le but est de visiter toutes les villes d'un graphe interconnecté de villes, et ce, en parcourant la plus petite distance possible.

***DEVOIR***:  
Parcourir le notebook, en exécutant chaque celle, une à une.  
Pour chaque **(TO DO)**, effectuer les tâches demandées.  
Quand vous avez terminé, signez (voir cette étape ajoutée à la fin) et soumettez votre notebook.
***

**1. Nous définissons d'abord un petit nombre de villes.**

In [1]:
# let's define 5 cities with a label and a (x,y) coordinate
cities = {'A': (0,0), 'B': (1,1), 'C': (1,0), 'D': (-1,1), 'E': (-1,-1)}

La méthode *showNodes* ci-bas nous permet de visualiser la position des villes.
Les packages **matplotlib** et **networkx** doivent d'abord être installés. 
Pour faire l'installation des packages, ouvrir une invite de commande, et tapez **pip install matplotlib**, ainsi que **pip install networkx**.  Une fois les packages installés, vous devrez probablement redémarrer le *kernel*.  Ensuite, vous serez en mesure d'exécuter la cellule ci-bas.

In [2]:
import matplotlib.pyplot as plt
import networkx as nx

# small method to show nodes
# provide defaults for figure size (size), as well as x and y axis
def showNodes(nodeDict, size= (6.5, 4), xLim=[-2,2], yLim=[-2,2]):
    fig = plt.figure(figsize=size)
    axg = fig.add_subplot(111)
    axg.set_xlim(xLim)
    axg.set_ylim(yLim)

    G=nx.Graph()
    for k in nodeDict.keys():
        G.add_node(k, coor=nodeDict.get(k))
    
    pos=nx.get_node_attributes(G,'coor')
    nx.draw(G, pos, with_labels = True)
    plt.axis('on')
    plt.show()

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# show the cities on a map
showNodes(cities)

#### 2. Mesurons les distances entre les villes.
D'abord, nous définissons une méthode pour calculer la distance entre deux villes spécifiques.  Ensuite nous définissons une méthode pour trouver l'ensemble des distances entre plusieurs paires de noeuds (villes).

In [ ]:
import math

# returns the euclidian distance between two nodes
def dist(node1, node2):
    return round(math.sqrt(math.pow(node1[0] - node2[0], 2) + math.pow(node1[1] - node2[1], 2)), 3)

In [ ]:
# Small test to see the distance
dist(cities.get('A'), cities.get('B'))

In [ ]:
# building a dictionary of all pairs or ditances
def distDict(nodes):
    distance = {}
    # go through all pairs of nodes
    for k in nodes.keys():
        for m in nodes.keys():
            distance[k,m] = dist(nodes.get(k), nodes.get(m))
    return distance

In [ ]:
# showing the distances between the cities
distances = distDict(cities)
print(distances)

Ajoutons aussi une méthode pour trouver la ville la plus proche d'une autre ville.  Cette méthode sera utile plus tard.

In [ ]:
# find the min distance from node1 to all other nodes.
# it receives the node1 label, the labels of other nodes to test,
# and the precalculated distances between all nodes
def closestNode(node1_label, otherNodes_Labels, distanceDict):
    # tricky syntax... it creates tuples where the distance is the first element of the tuple
    # and the position in the array is the second element of the tuple
    minDist = min( (distanceDict[node1_label,k], k) for k in otherNodes_Labels)
    # get the second element of the tuple which is the index found
    foundNode = minDist[1]
    return foundNode

In [ ]:
# Testing the closest node
closestNode('B', ['C', 'D'], distances)

#### 3. Solution gloutonne d'abord.
Avant de pouvoir utiliser le *recuit simulé*, il faut une première solution.  Nous utilisons un algorithme glouton pour générer cette solution sous-optimale. 

In [ ]:
# greedy solution
import random

def greedy_solution(nodes):
    # distance matrix
    allDist = distDict(nodes)
    # pick a first node at random among the set of keys (node names)
    cur_node = random.choice(list(nodes.keys()))
    # add that node to solution
    solution = [cur_node]
    # build a free list of nodes (not yet used) containing all nodes except the one just chosen 
    free_list = list(nodes.keys())
    free_list.remove(cur_node)
    
    # while there are still nodes to be attached
    while free_list:
        # find the closestNode
        found_node = closestNode(cur_node, free_list, allDist)
        cur_node = found_node
        free_list.remove(cur_node)
        solution.append(cur_node)

    return solution

In [ ]:
# Test obtaining a greedy solution
gs = greedy_solution(cities)
print(gs)

#### 4. Visualisation d'un chemin dans le graphe.  
Nous modifions légèrement la méthode *showNodes* vue précédemment pour devenir *showPath* et montrer un chemin dans un graphe dirigé. 

In [ ]:
# small method to show a path
# it receives an ordered list of nodes e.g. ['B','C','A', 'D', 'E']
# and a dictionary of all the nodes and their positions
def showPath(path, nodeDict, size= (6.5, 4), xLim=[-2,2], yLim=[-2,2]):
    fig = plt.figure(figsize=size)
    axg = fig.add_subplot(111)
    axg.set_xlim(xLim)
    axg.set_ylim(yLim)

    G=nx.DiGraph()
    for k in path:
        G.add_node(k, pos=nodeDict.get(k))
        
    for i in range(len(path) - 1):
        x = path[i]
        y = path[i + 1]
        G.add_edge(x,y)
        
    pos=nx.get_node_attributes(G,'pos')
    nx.draw(G, pos, with_labels = True, edge_color = 'b', arrows=True)   
    # nx.draw(G, pos, with_labels = True)
    plt.axis('on')
    plt.show()

Un test pour visualiser 2 solutions gloutonnes.

In [ ]:
# Testing to output the path chosen by the greedy solution
gs1 = greedy_solution(cities)
print(gs1)
showPath(gs1, cities)
gs2 = greedy_solution(cities)
print(gs2)
showPath(gs2, cities)

**(Q1 - TO DO)**  Executer le code ci-haut quelques fois, jusqu'à l'obtention de 2 solutions différentes. Combien de fois avez-vous dû exécuter le code ci-haut pour obtenir 2 solutions gloutonnes différentes?  Et quelles sont ces solutions?

Nb d'exécutions :  2

Chemins des 2 solutions générées : ['E', 'A', 'C', 'B', 'D']​, ['B', 'C', 'A', 'D', 'E']

**(Q2 - TO DO)** Completer la méthode ci-bas pour trouver la longueur total d'un chemin. 

In [ ]:
# (TO DO) totalDistance, which receives a path (list of node labels) and the dictionary of distances
def totalDist(path, distanceDict):
    total = 0
    for i in range(len(path) - 1):
        x = path[i]
        y = path[i+1]
        total += distanceDict[x,y]
    return round(total,3)

In [ ]:
# Test the total distance method
print(totalDist(gs1, distances))
print(totalDist(gs2, distances))

**(Q3 -- TO DO)** Ajouter 2 nouvelles villes (positionner les villes où vous voulez dans le graphe prédéfini de -2 à 2).  Donc, refaire la séquence d'étapes ci-haut (1) définir un graphe (qui sera *cities* avec 2 villes de plus), (2) visualiser les positions des villes (3) contruire le dictionnaire des distances (4) trouver deux solutions gloutonnes (5) montrer les chemins pour ces solutions. 

In [ ]:
# (1) defining a larger set of cities
moreCities = {'A': (0,0), 'B': (1,1), 'C': (1,0), 'D': (-1,1), 'E': (-1,-1), 'F' : (0,1), 'G' : (0,-1)}
# (2)
showNodes(moreCities)
# (3)
moreDistances = distDict(moreCities)
# (4)
more_gs1 = greedy_solution(moreCities)
more_gs2 = greedy_solution(moreCities)
# (5)
print(more_gs1)
showPath(more_gs1, moreCities)
print(more_gs2)
showPath(more_gs2, moreCities)
print(totalDist(more_gs1, moreDistances))
print(totalDist(more_gs2, moreDistances))

**5. Definissons le processus du recuit simulé**  
La classe ci-bas définit le processus du recuit simulé.  Cette classe est une adaptation de la classe *anneal.py* qui se trouve dans le code python du github https://github.com/chncyhn/simulated-annealing-tsp. Vous n'avez pas à modifier cette classe, mais tenter de la comprendre pour assimiler l'algorithme du recuit simulé. 

In [ ]:
# class to perform a simulated annealing
# must start with a set of nodes, all other parameters have defaults
class SimAnneal(object):
    def __init__(self, nodes, T=-1, alpha=-1, stopping_T=-1, stopping_iter=-1):
        # set of nodes
        self.nodes = nodes
        # number of nodes
        self.N = len(nodes)
        # set the temperature T to sqrt(N) if not specified
        self.T = math.sqrt(self.N) if T == -1 else T
        # set alpha (rate at which the temperature is decreased)
        self.alpha = 0.995 if alpha == -1 else alpha
        # set stopping temprature
        self.stopping_temperature = 0.00000001 if stopping_T == -1 else stopping_T
        # set stopping iteration
        self.stopping_iter = 100000 if stopping_iter == -1 else stopping_iter
        # start at iteration 1
        self.iteration = 1

        # calculate the distances
        self.allDist = distDict(nodes)
        
        # start with greedy solution
        self.cur_solution = greedy_solution(nodes)
        
        # so far, the best solution is the one we have
        self.best_solution = list(self.cur_solution)

        # calculate fitness of the current solution
        self.cur_fitness = self.fitness(self.cur_solution)
        # initial fitness for initial solution
        self.initial_fitness = self.cur_fitness
        # best fitness so far
        self.best_fitness = self.cur_fitness
        # build the fitness list as we explore solutions
        self.fitness_list = [self.cur_fitness]


    def fitness(self, sol):
        """ Objective value of a solution """
        return totalDist(sol, self.allDist)

    # acceptance propability
    def p_accept(self, candidate_fitness):
        """
        Probability of accepting if the candidate is worse than current
        Depends on the current temperature and difference between candidate and current
        """
        return math.exp(-abs(candidate_fitness - self.cur_fitness) / self.T)

    def accept(self, candidate):
        """
        Accept with probability 1 if candidate is better than current
        Accept with probabilty p_accept(..) if candidate is worse
        """
        candidate_fitness = self.fitness(candidate)
        # test if fitness is smaller then the current one
        if candidate_fitness < self.cur_fitness:
            self.cur_fitness = candidate_fitness
            self.cur_solution = candidate
            # test if fitness is better than best so far, keep it as best if so
            if candidate_fitness < self.best_fitness:
                self.best_fitness = candidate_fitness
                self.best_solution = candidate

        # if solution is worst than the current one, there is still a possibility to pursue it
        else:
            if random.random() < self.p_accept(candidate_fitness):
                self.cur_fitness = candidate_fitness
                self.cur_solution = candidate

    def anneal(self):
        """
        Execute simulated annealing algorithm
        """
        # go through the temperature schedule
        print(self.T)
        print(self.stopping_temperature)
        while (self.T >= self.stopping_temperature) and (self.iteration < self.stopping_iter):
            # explore current solution
            candidate = list(self.cur_solution)
            # modify the solution to switch two cities
            # first chosen city between [2,N-1] - not changing the first city
            l = random.randint(2, self.N - 1)
            # second city between [0,N-1] Not sure why ??
            i = random.randint(0, self.N - l)
            # the chosen nodes are switch
            candidate[i:(i + l)] = reversed(candidate[i:(i + l)])
            # perform acceptance test 
            self.accept(candidate)
            # multiply the temperature by alpha (this will reduce it)
            self.T *= self.alpha
            self.iteration += 1
            
            # add current fitness to list (to be able to display)
            self.fitness_list.append(self.cur_fitness)

        print('Best fitness obtained: ', self.best_fitness)
        print('Improvement over greedy heuristic: ',
              round((self.initial_fitness - self.best_fitness) / (self.initial_fitness), 4))

        return self.best_solution
    
    def plot_learning(self):
        """
        Plot the fitness through iterations
        """
        plt.plot([i for i in range(len(self.fitness_list))], self.fitness_list)
        plt.ylabel('Fitness')
        plt.xlabel('Iteration')
        plt.show()


#### 6. Refinement d'une solution gloutonne avec recuit simulé.

**(Q4 - TO DO)** Commentez chaque ligne du code ci-bas pour expliquer ce qui se passe.  Aussi exécuter le code quelques fois.  Cela vous montrera qu'il peut y avoir parfois un coût total plus bas en appliquant le recuit simulé.

In [ ]:
# Executes the greedy solution algorithm on a list of cities which is used to compare the solutions produced by SimAnneal
firstSol = greedy_solution(cities)

# Creates a dictionary of distances between all cities
distances = distDict(cities)

# Prints the greedy solution path and the total distance travelled using it
print('Solution {} takes a total of {}'.format(firstSol,totalDist(firstSol, distances)))

# Prints the path (produced by the greedy solution) on a graph
showPath(firstSol, cities)

# Initializes SimAnneal with the same list of cities
# Sets the initial temperature, cooling rate, maximum number of iterations and stopping temperature to default values unless specified otherwise
sa = SimAnneal(cities)

# Executes the SimAnneal algorithm
# Starts with a basic solution (provided by a greedy solution in this case) and attempts to improve it by swapping its nodes randomly
# If it has a better fitness rating, it is kept as the current best solution
# Stops when the temperature reaches the lower bound or if the number of iterations reaches the set maximum
betterSol = sa.anneal()

# Prints the path (produced by the SimAnneal solution) on a graph
showPath(betterSol, cities)

**7. Test à plus grande échelle**  
Le fichier *coord.txt* (que vous trouverez aussi sur Brightspace dans la section des Jupyter Notebooks), contient un ensemble de coordonnées. Ce fichier de données est aussi tiré du github https://github.com/chncyhn/simulated-annealing-tsp.  Voici le début du fichier.

1 0 0  
2 3 5  
3 2.5 9  
4 48 16  
5 48 17  
6 69 16  
...  

Nous utiliserons le chiffre de la première colonne comme étiquette de la ville (e.g. ville 0, ville 1, ...) et les deux autres colonnes comme les coordonnées.

Assurez-vous de mettre le fichier *coord.txt* dans le même répertoire que votre notebook (ou sinon changer le chemin d'accès dans le code ci-bas).

In [ ]:
# reading the coordinates
manyCities = {}
with open('coord.txt','r') as f:
    i = 0
    for line in f.readlines():
        line = [float(x.replace('\n','')) for x in line.split(' ')]
        label = str(int(line[0]))
        manyCities[label] = (line[1], line[2])
        i += 1

Visualisons où sont les noeuds pour cet exemple.

In [ ]:
showNodes(manyCities, size=(15,15), xLim=[-75,100], yLim=[-110,60])

Nous pouvons maintenant tester le processus de *recuit simulé* sur ce plus grand exemple.  Le code test est très similaire au code que vous avez commenté à la question (4) ci-haut.  Regarder l'étape supplémentaire, *plot_learning*, qui montre la valeur de la fonction de coût au fur et à mesure de la progression de l'algorithme. 

In [ ]:
firstSol = greedy_solution(manyCities)
distances = distDict(manyCities)
print('Solution {} takes a total of {}'.format(firstSol,totalDist(firstSol, distances)))
showPath(firstSol, manyCities, size=(15,15), xLim=[-75,100], yLim=[-110,60])
sa = SimAnneal(manyCities)
betterSol = sa.anneal()
sa.plot_learning()
showPath(betterSol, manyCities, size=(15,15), xLim=[-75,100], yLim=[-110,60])

**(Q5 - TO DO)** Tester le recuit simulé en variant certains paramètres, tel en faisant un changement sur les températures à explorer (T, stopping_T, alpha), ou encore sur le nombre d'itérations maximales à faire.  Ces paramètres sont tous des paramètres possibles à assigner à l'instantiation de la classe SimAnneal.  Faites 3-4 explorations et indiquer ce que vous tester.

In [ ]:
# (TO DO) Look at this line from the SimAnneal class above
# def __init__(self, nodes, T=-1, alpha=-1, stopping_T=-1, stopping_iter=-1):
# This means that parameters T, alpha, stopping_T and stopping_iter all have defaults, but could be changed
# For example, calling SimAnneal(manyCities, stopping_iter=100) means that I set a maximum of 100 iterations.
# Try a few variations and look at the differences

# Test 1: Faster temperature decrease
sa = SimAnneal(manyCities, -1, 0.1, -1, -1)
betterSol = sa.anneal()
sa.plot_learning()
showPath(betterSol, manyCities, size=(15,15), xLim=[-75,100], yLim=[-110,60])
# Generally too fast to be able to retrieve better solutions

# Test 2: Higher starting temperature
sa = SimAnneal(manyCities, len(manyCities))
betterSol = sa.anneal()
sa.plot_learning()
showPath(betterSol, manyCities, size=(15,15), xLim=[-75,100], yLim=[-110,60])


# Test 3: Reduce number of iterations
sa = SimAnneal(manyCities, stopping_iter=100)
betterSol = sa.anneal()
sa.plot_learning()
showPath(betterSol, manyCities, size=(15,15), xLim=[-75,100], yLim=[-110,60])
# Shorter running times don't allow the algorithm to produce many better solutions

**(Optionel - TO DO)**  Si vous vous intéressé à l'analyse de performance, vous pouvez tester une approche de solution optimale (telle la programmation dynamique) qui pourra explorer tout l'espace de solutions et trouver la solution optimale.  Une fois que vous avez cette solution, vous pourrez alors exécuter le *recuit simulé* 1000 fois et calculer le nombre de fois où la solution approximative s'est avéré être (ou être proche de) la solution optimale.  Faites rouler aussi 1000 fois la solution gloutonne et faites le même calcul.  Quelle solution semble la meilleure?

#### Signature

Je, Oliver Scott, declare que les réponses inscrites dans ce notebook sont les miennes.